In [1]:
import pandas as pd

In [2]:
import extraccion_de_archivos
import funciones_transformacion

In [3]:
sucursal_df,productos_df,precios_semana_20200413_df,precios_20200419_20200419_df,precios_20200426_20200426_df,precios_semana_20200503_df,precios_semana_20200518_df=funciones_transformacion.transformar_todos_los_archivos()

In [4]:
sucursal_df.columns

Index(['sucursal_id', 'comercioId', 'banderaId', 'banderaDescripcion',
       'comercioRazonSocial', 'provincia', 'localidad', 'direccion', 'lat',
       'lng', 'sucursalNombre', 'sucursalTipo'],
      dtype='object')

In [5]:
productos_df.columns

Index(['producto_id', 'marca', 'nombre', 'presentacion', 'categoria1',
       'categoria2', 'categoria3'],
      dtype='object')

In [8]:
import sqlite3

# Conectar a la base de datos (si no existe, se creará)
conn = sqlite3.connect('mi_base_de_datos.db')

# Crear un objeto cursor para ejecutar comandos SQL
cursor = conn.cursor()

# Comando SQL para crear una tabla llamada 'sucursal' con dos columnas de IDs y una columna de precios
cursor.execute('''DROP TABLE IF EXISTS sucursal;''')
create_table_query = '''
CREATE TABLE sucursal (
    sucursal_id TEXT NOT NULL PRIMARY KEY,
    comercioId INTEGER NOT NULL,
	banderaId INTEGER NOT NULL,
    banderaDescripcion TEXT,
    comercioRazonSocial	TEXT,
    provincia TEXT,
    localidad TEXT
    direccion TEXT
    lat	TEXT
    lng	TEXT
    sucursalNombre TEXT
    sucursalTipo TEXT
    FOREIGN KEY(sucursal_id) REFERENCES sucursal(sucursal_id)
);
'''
print("Tabla 'sucursal' creada exitosamente.")
#borrar tabla precio si existe
cursor.execute('''DROP TABLE IF EXISTS precio;''')

# Comando SQL para crear una tabla llamada 'precio' con dos columnas de IDs y una columna de precios
create_table_query = '''
CREATE TABLE precio (
    sucursal_id TEXT NOT NULL,
    producto_id INTEGER NOT NULL,
    precio REAL,
    FOREIGN KEY(sucursal_id) REFERENCES sucursal(sucursal_id),
    FOREIGN KEY(producto_id) REFERENCES sucursal(producto_id)
);
'''
print("Tabla 'precio' creada exitosamente.")
#borrar tabla precio si existe
cursor.execute('''DROP TABLE IF EXISTS producto;''')
# Comando SQL para crear una tabla llamada 'producto' con dos columnas de IDs y una columna de precios
create_table_query = '''
CREATE TABLE producto (
    producto_id INTEGER NOT NULL PRIMARY KEY,
    marca TEXT,
    nombre TEXT,
    presentacion TEXT,
    categoria1 TEXT,
    categoria2 TEXT,
    categoria3 TEXT
);
'''

# Ejecutar el comando SQL para crear la tabla
cursor.execute(create_table_query)

# Guardar los cambios y cerrar la conexión


print("Tabla 'producto' creada exitosamente.")


Tabla 'sucursal' creada exitosamente.
Tabla 'precio' creada exitosamente.
Tabla 'producto' creada exitosamente.


In [7]:
precios_semana_20200413_df.shape

(472151, 3)

In [9]:
# Insertar o actualizar los datos en la tabla precio
sucursal_df.to_sql('sucursal', conn, if_exists='replace', index=False, index_label='sucursal_id')

# Guardar los cambios 
conn.commit()

In [11]:
cursor.execute('SELECT COUNT(*) FROM sucursal')

# Obtener el resultado de la consulta
cantidad_registros = cursor.fetchone()[0]
print("En la tabla sucursal se insertaron",cantidad_registros)

En la tabla sucursal se insertaron 2333


In [12]:
# Insertar o actualizar los datos en la tabla precio
productos_df.to_sql('producto', conn, if_exists='replace', index=False, index_label='sucursal_id')

# Guardar los cambios 
conn.commit()

In [13]:
cursor.execute('SELECT COUNT(*) FROM producto')

# Obtener el resultado de la consulta
cantidad_registros = cursor.fetchone()[0]
print("En la tabla sucursal se insertaron",cantidad_registros)

En la tabla sucursal se insertaron 72038


inserto o actualizo los registros en la tabla precio los siguintes archivos:
precios_semana_20200413_df,
precios_20200419_20200419_df,
precios_20200426_20200426_df,
precios_semana_20200503_df,
precios_semana_20200518_df

In [63]:
# Insertar o actualizar los datos en la tabla precio
precios_semana_20200413_df.to_sql('precio', conn, if_exists='replace', index=False, index_label='sucursal_id')

# Guardar los cambios 
conn.commit()


In [9]:
cursor.execute('SELECT COUNT(*) FROM precio')

# Obtener el resultado de la consulta
cantidad_registros = cursor.fetchone()[0]
print("En la tabla precio se insertaron",cantidad_registros)

En la tabla precio se insertaron 472151


In [64]:
precios_actualizados=pd.concat([precios_semana_20200413_df,precios_20200419_20200419_df]).drop_duplicates(subset =[ 'sucursal_id','producto_id'],keep='first').reset_index(drop=True)

In [59]:
precios_actualizados.shape

(760463, 3)

In [65]:
# Insertar o actualizar los datos en la tabla precio
precios_actualizados.to_sql('precio', conn, if_exists='replace', index=False, index_label='sucursal_id')

# Guardar los cambios 
conn.commit()


In [11]:
precios_20200419_20200419_df.shape


(444272, 3)

In [66]:
cursor.execute('SELECT COUNT(*) FROM precio')

# Obtener el resultado de la consulta
cantidad_registros = cursor.fetchone()[0]
print("En la tabla precio se insertaron",cantidad_registros)

En la tabla precio se insertaron 760463


In [69]:
#consula selecciono todo de la tabla precio
query = 'SELECT * FROM precio'
#almaceno en df
df_precios_antiguos = pd.read_sql(query, conn)
#creo el dataframe precios_actualizados
precios_actualizados=pd.DataFrame()
#borro los primeros registros duplicados
precios_actualizados=pd.concat([df_precios_antiguos,precios_20200426_20200426_df]).drop_duplicates(subset =[ 'sucursal_id','producto_id'],keep='first').reset_index(drop=True)
# Insertar o actualizar los datos en la tabla precio
precios_actualizados.to_sql('precio',  conn, if_exists='replace', index=False, index_label='sucursal_id')

# Guardar los cambios 
conn.commit()

In [72]:
precios_actualizados.shape

(959302, 3)

In [71]:

#consula selecciono todo de la tabla precio
query = 'SELECT * FROM precio'
#almaceno en df
df_precios_antiguos = pd.read_sql(query, conn)
#creo el dataframe precios_actualizados
precios_actualizados=pd.DataFrame()
#borro los primeros registros duplicados
precios_actualizados=pd.concat([df_precios_antiguos,precios_semana_20200503_df]).drop_duplicates(subset =[ 'sucursal_id','producto_id'],keep='first').reset_index(drop=True)
# Insertar o actualizar los datos en la tabla precio
precios_actualizados.to_sql('precio',  conn, if_exists='replace', index=False, index_label='sucursal_id')

# Guardar los cambios 
conn.commit()

In [73]:
#consula selecciono todo de la tabla precio
query = 'SELECT * FROM precio'
#almaceno en df
df_precios_antiguos = pd.read_sql(query, conn)
#creo el dataframe precios_actualizados
precios_actualizados=pd.DataFrame()
#borro los primeros registros duplicados
precios_actualizados=pd.concat([df_precios_antiguos,precios_semana_20200518_df]).drop_duplicates(subset =[ 'sucursal_id','producto_id'],keep='first').reset_index(drop=True)
# Insertar o actualizar los datos en la tabla precio
precios_actualizados.to_sql('precio',  conn, if_exists='replace', index=False, index_label='sucursal_id')

# Guardar los cambios 
conn.commit()

In [74]:
cursor.execute('SELECT COUNT(*) FROM precio')

# Obtener el resultado de la consulta
cantidad_registros = cursor.fetchone()[0]
print("En la tabla precio se insertaron registros totales",cantidad_registros)

En la tabla precio se insertaron registros totales 1139820


In [75]:
cursor.execute('SELECT COUNT(*) FROM (SELECT DISTINCT sucursal_id,producto_id FROM precio) as unique_combinations')

# Obtener el resultado de la consulta
cantidad_registros = cursor.fetchone()[0]
print("En la tabla precio se insertaron distintos",cantidad_registros)

En la tabla precio se insertaron distintos 1139820


In [ ]:
conn.commit()
conn.close()